## Part 2. PySpark & HDFS Experiment

### PySpark Session and HDFS connection

In [1]:
# Import PySpark and initialize Spark session
import pyspark
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("PySparkHDFS") \
    .config("spark.jars", "../lib/mysql-connector-j-8.2.0.jar") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9000") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/09 12:41:39 WARN Utils: Your hostname, Xuan-Nguyet-Nguyen.local, resolves to a loopback address: 127.0.0.1; using 192.168.1.231 instead (on interface en0)
25/12/09 12:41:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/12/09 12:41:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Exercise 1. Load all tables into DataFrames and write to HDFS

In [2]:
# Get all schema information
tables = (
    spark.read.format("jdbc") \
        .option("url", "jdbc:mysql://localhost:3306/sakila") \
        .option("driver", "com.mysql.cj.jdbc.Driver") \
        .option("dbtable", "(SELECT table_name FROM information_schema.tables WHERE table_schema='sakila') AS t") \
        .option("user", "root") \
        .option("password", "rootpassword") \
        .load()
)

In [3]:
tables.collect()

[Row(table_name='payment'),
 Row(table_name='store'),
 Row(table_name='film'),
 Row(table_name='staff_list'),
 Row(table_name='customer'),
 Row(table_name='customer_list'),
 Row(table_name='film_actor'),
 Row(table_name='sales_by_store'),
 Row(table_name='film_text'),
 Row(table_name='inventory'),
 Row(table_name='actor'),
 Row(table_name='nicer_but_slower_film_list'),
 Row(table_name='sales_by_film_category'),
 Row(table_name='actor_info'),
 Row(table_name='language'),
 Row(table_name='staff'),
 Row(table_name='address'),
 Row(table_name='rental'),
 Row(table_name='country'),
 Row(table_name='city'),
 Row(table_name='category'),
 Row(table_name='film_list'),
 Row(table_name='film_category')]

In [4]:
table_names = [row.table_name for row in tables.collect()]
table_names


['payment',
 'store',
 'film',
 'staff_list',
 'customer',
 'customer_list',
 'film_actor',
 'sales_by_store',
 'film_text',
 'inventory',
 'actor',
 'nicer_but_slower_film_list',
 'sales_by_film_category',
 'actor_info',
 'language',
 'staff',
 'address',
 'rental',
 'country',
 'city',
 'category',
 'film_list',
 'film_category']

In [ ]:
# Try to import all tables
for table_name in table_names:
    df = spark.read.format("jdbc") \
        .option("url", "jdbc:mysql://localhost:3306/sakila") \
        .option("driver", "com.mysql.cj.jdbc.Driver") \
        .option("dbtable", table_name) \
        .option("user", "root") \
        .option("password", "rootpassword") \
        .load()
    print(table_name)
    df.show(5)

payment
+----------+-----------+--------+---------+------+-------------------+-------------------+
|payment_id|customer_id|staff_id|rental_id|amount|       payment_date|        last_update|
+----------+-----------+--------+---------+------+-------------------+-------------------+
|         1|          1|       1|       76|  2.99|2005-05-25 11:30:37|2006-02-15 22:12:30|
|         2|          1|       1|      573|  0.99|2005-05-28 10:35:23|2006-02-15 22:12:30|
|         3|          1|       1|     1185|  5.99|2005-06-15 00:54:12|2006-02-15 22:12:30|
|         4|          1|       2|     1422|  0.99|2005-06-15 18:02:53|2006-02-15 22:12:30|
|         5|          1|       2|     1476|  9.99|2005-06-15 21:08:46|2006-02-15 22:12:30|
+----------+-----------+--------+---------+------+-------------------+-------------------+
only showing top 5 rows
store
+--------+----------------+----------+-------------------+
|store_id|manager_staff_id|address_id|        last_update|
+--------+---------------

+--------+----------+------------+--------------------+
|actor_id|first_name|   last_name|           film_info|
+--------+----------+------------+--------------------+
|       1|  PENELOPE|     GUINESS|Animation: ANACON...|
|       2|      NICK|    WAHLBERG|Action: BULL SHAW...|
|       3|        ED|       CHASE|Action: CADDYSHAC...|
|       4|  JENNIFER|       DAVIS|Action: BAREFOOT ...|
|       5|    JOHNNY|LOLLOBRIGIDA|Action: AMADEUS H...|
+--------+----------+------------+--------------------+
only showing top 5 rows
language
+-----------+--------------------+-------------------+
|language_id|                name|        last_update|
+-----------+--------------------+-------------------+
|          1|English             |2006-02-15 05:02:19|
|          2|Italian             |2006-02-15 05:02:19|
|          3|Japanese            |2006-02-15 05:02:19|
|          4|Mandarin            |2006-02-15 05:02:19|
|          5|French              |2006-02-15 05:02:19|
+-----------+----------

In [ ]:
# # Load all tables into DataFrames and write to HDFS
# for table_name in table_names:
#     df = spark.read.format("jdbc") \
#         .option("url", "jdbc:mysql://localhost:3306/sakila") \
#         .option("driver", "com.mysql.cj.jdbc.Driver") \
#         .option("dbtable", table_name) \
#         .option("user", "root") \
#         .option("password", "rootpassword") \
#         .load()
#     df.write.mode("overwrite").parquet(f"hdfs://localhost:9000/user/hadoop/sakila/{table_name}")

25/12/09 12:44:13 WARN DataStreamer: Exception in createBlockOutputStream blk_1073741825_1001
org.apache.hadoop.net.ConnectTimeoutException: 60000 millis timeout while waiting for channel to be ready for connect. ch : java.nio.channels.SocketChannel[connection-pending remote=/172.18.0.5:50010]
	at org.apache.hadoop.net.NetUtils.connect(NetUtils.java:617)
	at org.apache.hadoop.hdfs.DataStreamer.createSocketForPipeline(DataStreamer.java:256)
	at org.apache.hadoop.hdfs.DataStreamer.createBlockOutputStream(DataStreamer.java:1894)
	at org.apache.hadoop.hdfs.DataStreamer.setupPipelineForCreate(DataStreamer.java:1842)
	at org.apache.hadoop.hdfs.DataStreamer.run(DataStreamer.java:752)
25/12/09 12:44:13 WARN DataStreamer: Abandoning BP-1723281904-127.0.0.1-1764932735654:blk_1073741825_1001
25/12/09 12:44:13 WARN DataStreamer: Excluding datanode DatanodeInfoWithStorage[172.18.0.5:50010,DS-8f7725ad-e4ad-445d-a44c-03d59c0f6684,DISK]
25/12/09 12:44:13 WARN DataStreamer: DataStreamer Exception
org.a

Py4JJavaError: An error occurred while calling o52.parquet.
: org.apache.spark.SparkException: [TASK_WRITE_FAILED] Task failed while writing rows to hdfs://localhost:9000/user/hadoop/sakila/payment. SQLSTATE: 58030
	at org.apache.spark.sql.errors.QueryExecutionErrors$.taskFailedWhileWritingRowsError(QueryExecutionErrors.scala:772)
	at org.apache.spark.sql.execution.datasources.FileFormatDataWriter.enrichWriteError(FileFormatDataWriter.scala:92)
	at org.apache.spark.sql.execution.datasources.FileFormatDataWriter.commit(FileFormatDataWriter.scala:123)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.$anonfun$executeTask$1(FileFormatWriter.scala:407)
	at org.apache.spark.util.Utils$.tryWithSafeFinallyAndFailureCallbacks(Utils.scala:1323)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.executeTask(FileFormatWriter.scala:418)
	at org.apache.spark.sql.execution.datasources.WriteFilesExec.$anonfun$doExecuteWrite$1(WriteFiles.scala:107)
	at org.apache.spark.rdd